# Interactive 2D sectional-Voronoi spiderweb

By Mark Neyrinck and Johan Hidding (whose algorithm in https://github.com/jhidding/adhesion-example this is based on). See also https://arxiv.org/abs/1710.04509, Whiteley et al., https://link.springer.com/chapter/10.1007/978-0-387-92714-5_18, and articles by Robert Lang about spiderwebs and origami tessellations, e.g. "Every Spider Web Has a Simple Flat Twist Tessellation", https://books.google.com/books?id=mthYCwAAQBAJ&pg=PA190, and https://books.google.com/books?hl=en&lr=&id=r-k4GSYaV5YC&oi=fnd&pg=PA455.

Sectional-Voronoi diagrams (also known as power diagrams and additively-weighted Voronoi diagrams) are "spiderwebs", a structural-engineering term for a spatial graph that can be strung up to be entirely either in tension, or compression.  A Voronoi tessellation is a set of cells, each cell of which is the patch of space closest to its correpsonding generator. In a sectional-Voronoi tessellation, there is a constant added in quadrature to the distance function (a.k.a. "power function") used for each generator, generally different for each generator. This constant can be interpreted as a distance perpendicular to the space being tessellated, thus the tessellation can be interpreted as a "section" through a higher-dimensional space.

There are two figures below. Fig 2 shows blue dots (2D positions of generators) and black lines (sectional-Voronoi edgges). If Fig 1 remains untouched, Fig 2 is a usual Voronoi tessellation. But generators (green, in Fig 1) can be slid vertically to adjust their potential, i.e. additive weight, or "power", sliding Voronoi edges in or out in Fig 2, producing a sectional-Voronoi diagram.

To add: Delaunay visualization, representing tensions on each edge


In [1]:
import numpy as np, pylab as plt
import bqplot # Bloomberg plotting package with straightforward interactivity
import sectional_tess #package in this repository with sectional tessellation code
#from scipy.spatial import Voronoi, Delaunay, voronoi_plot_2d

In [2]:
# Set initial positions of generators

# 5x5 grid
# x_data = np.repeat(np.arange(5),5)
# y_data = np.tile(np.arange(5),5)

# Concentric circles:
# Outernum = #generators along an outer circle of radius 2
# Innernum is #generators along a circle of radius 1
# Default (outernum,innernum)=(3,2) produces a framework roughly resembling the Eiffel Tower
outernum = 3
innernum = 2
y_data = -np.concatenate((2.*np.cos(2.*np.pi/float(outernum)*np.arange(outernum)),
                         1.*np.cos(2.*np.pi*np.arange(innernum)/float(outernum))))+2.
x_data = np.concatenate((2.*np.sin(2.*np.pi/float(outernum)*np.arange(outernum)),
                        1.*np.sin(2.*np.pi*np.arange(innernum)/float(innernum))))+2.

In [3]:
## User interface based on https://githubqplot.com/bloomberg/bqplot/blob/master/examples/Marks/Scatter.ipynb
sc_x = bqplot.LinearScale(stabilized=True,max=5,min=-1)
sc_y = bqplot.LinearScale(stabilized=True,max=5,min=-1)

scat_height = bqplot.Scatter(x=x_data, y=y_data, scales={'x': sc_x, 'y': sc_y}, colors=['green'],
               enable_move=True, restrict_y=True)
scat_height.y_data_init = 1.*y_data
scat = bqplot.Scatter(x=x_data, y=y_data, scales={'x': sc_x, 'y': sc_y}, colors=['blue'],
               enable_move=True)

lin = bqplot.Lines(x=[], y=[], scales={'x': sc_x, 'y': sc_y}, colors=['black'])
lin_ext = bqplot.Lines(x=[], y=[], scales={'x': sc_x, 'y': sc_y}, colors=['black'])

def update_line(change=None):
    with lin.hold_sync():\
        
        # if a point was added to scat
        if (len(scat.y) == len(scat_height.y) + 1):
            scat_height.y = np.append(scat_height.y, scat.y[-1])
        if (len(scat.y) == len(scat_height.y_data_init) + 1):
            scat_height.y_data_init = np.append(scat_height.y_data_init, scat.y[-1])
        if (len(scat.x) == len(scat_height.x) + 1):
            scat_height.x = np.append(scat_height.x, scat.x[-1])            
            
        # if a point was added to scat_height
        if (len(scat_height.y) == len(scat.y) + 1):
            scat.y = np.append(scat.y, scat_height.y[-1])  
        if (len(scat_height.y) == len(scat_height.y_data_init) + 1):
            scat_height.y_data_init = np.append(scat_height.y_data_init,scat_height.y[-1])
        if (len(scat_height.x) == len(scat.x) + 1):
            scat.x = np.append(scat.x, scat_height.x[-1])               
        
        # calculate sectional voronoi diagram
        vor = sectional_tess.sectional_voronoi(np.transpose(np.array([scat.x,scat.y])),
                                               scat_height.y-scat_height.y_data_init)
        
        # The rest of update_line is based on scipy.spatial.voronoi_plot_2d
        lenridgevert = len(vor.ridge_vertices)
        lin.x = -np.ones(2*lenridgevert,dtype=np.float)
        lin.y = -np.ones(2*lenridgevert,dtype=np.float)
        lin_ext.x = -np.ones(2*lenridgevert,dtype=np.float)
        lin_ext.y = -np.ones(2*lenridgevert,dtype=np.float)
        counter2 = 0
        for isimplex in range(lenridgevert):
            #print vor.ridge_vertices[isimplex]
            simplex = np.asarray(vor.ridge_vertices[isimplex])
            if np.all(simplex >= 0):
                #print simplex
                lin.x[counter2:counter2+2]= vor.vertices[simplex][:,0]
                lin.y[counter2:counter2+2]= vor.vertices[simplex][:,1]
                counter2 += 2
        lin.x = lin.x[:counter2].reshape(counter2/2,2)
        lin.y = lin.y[:counter2].reshape(counter2/2,2)
                
        center = vor.points.mean(axis=0)
        external_scale = np.sqrt(np.std(scat.x)*np.std(scat.y))
        counter2 = 0
        for pointidx, simplex in zip(vor.ridge_points, vor.ridge_vertices):
            simplex = np.asarray(simplex)
            if np.any(simplex < 0):
                i = simplex[simplex >= 0][0]  # finite end Voronoi vertex

                t = vor.points[pointidx[1]] - vor.points[pointidx[0]]  # tangent
                normt = np.linalg.norm(t)
                if normt > 0.:
                    t /= normt
                n = np.array([-t[1], t[0]])  # normal

                midpoint = vor.points[pointidx].mean(axis=0)
                direction = np.sign(np.dot(midpoint - center, n)) * n
                far_point = vor.vertices[i] + direction*external_scale

                lin_ext.x[counter2:counter2+2]= [vor.vertices[i,0],far_point[0]]
                lin_ext.y[counter2:counter2+2]= [vor.vertices[i,1],far_point[1]]

                counter2 += 2

        lin_ext.x = lin_ext.x[:counter2].reshape(counter2/2,2)
        lin_ext.y = lin_ext.y[:counter2].reshape(counter2/2,2)

        
update_line()
# update line on change of x or y of scatter

scat_height.observe(update_line,names=['y'])

scat.observe(update_line, names=['x'])
scat.observe(update_line, names=['y'])

ax_x = bqplot.Axis(scale=sc_x)
ax_y = bqplot.Axis(scale=sc_y, orientation='vertical')

# change the bleow "with" statements to e.g. disable adding points
with scat_height.hold_sync():
    scat_height.update_on_move = True
    scat_height.update_on_add = True
    scat_height.interactions = {'click': 'add'}
#allow adding generators to 'scat_height' (Fig 1)

with scat.hold_sync():
    scat.update_on_move = True #dynamic update
    scat.update_on_add = True 
    scat.interactions = {'click': 'add'}
#allow adding generators to 'scat' (Fig 2)

Fig 1: Generators in green; slide vertically to adjust additive weights. 

Fig 2: 2D position of generators and sectional-Voronoi edges.

Generators can be added to either figure, but currently they cannot be deleted, and changes (moving or adding points) cannot be undone. Thus it can be useful to turn off point-adding.

In [4]:
bqplot.Figure(marks=[scat_height], axes=[ax_x, ax_y],min_aspect_ratio=1,max_aspect_ratio=1)

Figure(axes=[Axis(scale=LinearScale(max=5.0, min=-1.0, stabilized=True)), Axis(orientation='vertical', scale=LinearScale(max=5.0, min=-1.0, stabilized=True))], fig_margin={'top': 60, 'right': 60, 'bottom': 60, 'left': 60}, layout=Layout(min_width=u'125px'), marks=[Scatter(colors=['green'], enable_move=True, interactions={'click': 'add'}, restrict_y=True, scales={'y': LinearScale(max=5.0, min=-1.0, stabilized=True), 'x': LinearScale(max=5.0, min=-1.0, stabilized=True)}, scales_metadata={'opacity': {'dimension': 'opacity'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'skew': {'dimension': 'skew'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 2.        ,  3.73205081,  0.26794919,  2.        ,  2.        ]), y=array([ 0. ,  3. ,  3. ,  1. ,  2.5]))], max_aspect_ratio=1.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))

In [5]:
bqplot.Figure(marks=[scat, lin, lin_ext], axes=[ax_x, ax_y],min_aspect_ratio=1,max_aspect_ratio=1)

Figure(axes=[Axis(scale=LinearScale(max=5.0, min=-1.0, stabilized=True)), Axis(orientation='vertical', scale=LinearScale(max=5.0, min=-1.0, stabilized=True))], fig_margin={'top': 60, 'right': 60, 'bottom': 60, 'left': 60}, layout=Layout(min_width=u'125px'), marks=[Scatter(colors=['blue'], enable_move=True, interactions={'click': 'add'}, scales={'y': LinearScale(max=5.0, min=-1.0, stabilized=True), 'x': LinearScale(max=5.0, min=-1.0, stabilized=True)}, scales_metadata={'opacity': {'dimension': 'opacity'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'skew': {'dimension': 'skew'}, 'rotation': {'dimension': 'rotation'}, 'size': {'dimension': 'size'}}, tooltip_style={'opacity': 0.9}, update_on_move=True, x=array([ 2.        ,  3.73205081,  0.26794919,  2.        ,  2.        ]), y=array([ 0. ,  3. ,  3. ,  1. ,  2.5])), Lines(colors=['black'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(max=5.0, min=-1.0, stabilized=True), 'x': LinearScale(max=5.0, min=-1.0, stabilized=True)}, scales_metadata={'y': {'orientation': 'vertical', 'dimension': 'y'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([[ 2.        ,  0.84529946],
       [ 2.        ,  3.15470054],
       [-0.59807621,  0.84529946],
       [-0.59807621,  4.59807621],
       [ 0.84529946,  3.15470054],
       [ 4.59807621,  3.15470054]]), y=array([[ 5.75,  1.75],
       [ 5.75,  1.75],
       [ 0.5 ,  1.75],
       [ 0.5 ,  0.5 ],
       [ 1.75,  1.75],
       [ 0.5 ,  1.75]])), Lines(colors=['black'], interactions={'hover': 'tooltip'}, scales={'y': LinearScale(max=5.0, min=-1.0, stabilized=True), 'x': LinearScale(max=5.0, min=-1.0, stabilized=True)}, scales_metadata={'y': {'orientation': 'vertical', 'dimension': 'y'}, 'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'color': {'dimension': 'color'}}, tooltip_style={'opacity': 0.9}, x=array([[ 2.        ,  2.        ],
       [-0.59807621, -1.59100149],
       [ 4.59807621,  5.59100149]]), y=array([[ 5.75      ,  6.89653135],
       [ 0.5       , -0.07326568],
       [ 0.5       , -0.07326568]]))], max_aspect_ratio=1.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0))